In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [10]:
df = pd.read_csv('./tesla_stock.csv')
df.head()
print(df.shape)

FileNotFoundError: [Errno 2] No such file or directory: './tesla_stock.csv'

In [ ]:
df.columns

In [ ]:
from datetime import datetime

In [ ]:
df.Date = pd.to_datetime(df['Date'])

In [ ]:
df.Date.head()

In [ ]:
df.set_index = df.Date

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
plt.title('Tesla Stock Closing Prices in past 5 years')
plt.xlabel('Days')
plt.ylabel('Closing Price')
df.Close.plot(legend = True, figsize = (10,5))

In [ ]:
plt.title('Tesla Stock Volume trading in past 5 years')
plt.xlabel('Days')
plt.ylabel('Volume of stocks')
df.Volume.plot(legend = True, figsize = (10,5))

In [ ]:
# Smoothing the graph
# Ploting prices with rolling mean
df['Close_10'] = df['Close'].rolling(10).mean()
df['Close_50'] = df['Close'].rolling(50).mean()
df.head(12)

In [ ]:
ax = df.Close.plot(x = 'Date', y = 'Close', title = 'Tesla Close Price', figsize = (10,5))
df.Close_10.plot(x = 'Date', y = 'Close_10', color = 'r', ax = ax)
df.Close_50.plot(x = 'Date', y = 'Close_50', color = 'g', ax = ax)
plt.ioff()

In [ ]:
# Daily returns
df['Daily_return'] = df['Close'].pct_change()
df['Daily_return'].plot(figsize = (10,5), legend = True, linestyle = '--', marker = 'o')
plt.ioff()

# Maximum daily fluctuation in this stock is 15 %

In [ ]:
# Average daily returns
sns.distplot(df['Daily_return'], bins = 1000, color = 'g')

In [ ]:
df['Daily_return'].hist(bins = 100, figsize = (10,5))

# Daily fluctuation of stock normal distribution is between +-2 %

In [ ]:
ser = np.array(range(len(df)))
ind_series = pd.Series(ser)
len(ind_series)

In [ ]:
df.set_index = ind_series
df.head()

In [ ]:
df2 = df.drop(['Date','Close_10','Close_50'],axis = 1, errors = 'ignore')
df2.head()

In [ ]:
df2_pct = df2.pct_change()
df2_pct = pd.DataFrame(df2_pct)
df2_pct['Date'] = df['Date']
df2_pct.head()

In [ ]:
df2_pct['Close'].quantile(0.05)

In [ ]:
df2_pct['Close'].quantile(0.60)

In [ ]:
# -0.042 means that 95% of the times the worst daily Loss will not exceed 4.25%

In [ ]:
# Value at risk using Monte Carlo simulation
days = 365
dt = 1/365
mu = df2_pct.mean()['Close']
sigma = df2_pct.std()['Close']

In [ ]:
def monte_price(start,days,mu,sigma):
    price = np.zeros(days)
    price[0] = start
    shock = np.zeros(days)
    drift = np.zeros(days)
    
    for x in range(1,days):
        shock[x] = np.random.normal(loc = mu*dt, scale = sigma*np.sqrt(dt))
        drift[x] = mu*dt
        price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))
    return price

In [ ]:
df.head()

In [ ]:
start = 29.959999
days = 50
for i in range(10000):
    plt.plot(monte_price(start,days,mu,sigma))

plt.xlabel('Days')
plt.ylabel('Close Price')
plt.title('Monte carlo simulation for Tesla')
plt.ioff()

In [ ]:
runs = 10000
simulations = np.zeros(runs)

for run in range(runs):
    simulations[run] = monte_price(start,days, mu, sigma)[days - 1]

In [ ]:
q = np.percentile(simulations,1)
sns.histplot(simulations)

# starting price
plt.figtext(0.6,0.8, s="Start price: $%.2f" %start)

# Mean close price
plt.axvline(x = simulations.mean(), linewidth = 2, color = 'g')
plt.figtext(0.6,0.7, s="Mean close price: $%.2f" %simulations.mean())

# Variance of price within 99% confidence level
plt.figtext(0.6,0.6, s="Var(0.99): $%.2f" %(start - q,))

# 1% quantile result
plt.figtext(0.15, 0.6, "q(0.99): $%.2f" % q)
plt.axvline(x = q, linewidth = 4, color = 'r')

plt.title('Tesla Closing price distribution after 365 days', weight = 'bold')

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test
        
        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(df.Close, lags=60)

In [ ]:
close_diff = df.Close - df.Close.shift(24)
tsplot(close_diff[24:], lags=60)

In [ ]:
close_diff = close_diff - close_diff.shift(1)
tsplot(close_diff[24+1:], lags=60)

In [ ]:
from tqdm import tqdm_notebook
from itertools import product 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ps = range(4, 8)
d=1 
qs = range(5, 8)
Ps = range(0, 2)
D=1 
Qs = range(0, 2)
s = 24 #season length is still 24

# creating list with all the possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimize_sarima(parameters_list,d,D,s):
    results = []
    best_aic = float('inf')
    
    for param in tqdm_notebook(parameters_list):
        try:
            model = sm.tsa.statespace.SARIMAX(df.Close, order=(param[0],d,param[1]),
                                              seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_params = param
        results.append([param, model.aic])
    
    result_table = pd.DataFrame(results)
    result_table.columns = ['Parameters','aic']
    result_table = result_table.sort_values(by='aic',ascending = True).reset_index(drop=True)
    return result_table

In [ ]:
%%time
result_table = optimize_sarima(parameters_list, d, D, s)

In [ ]:
result_table.head()

In [ ]:
p, q, P, Q = result_table.Parameters[0]

best_model=sm.tsa.statespace.SARIMAX(df.Close, order=(p, d, q), 
                                        seasonal_order=(P, D, Q, s)).fit(disp=-1)
print(best_model.summary())

In [ ]:
tsplot(best_model.resid[24+1:], lags=60)

In [ ]:
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def plotSARIMA(series, model, n_steps):
    # adding model values
    data = pd.Series(series.copy())
    dataframe = pd.DataFrame(data)
    dataframe.columns = ['actual']
    dataframe['arima_model'] = model.fittedvalues
    # making a shift on s+d steps, because these values were unobserved by the model
    # due to the differentiating
    dataframe['arima_model'][:s+d] = np.NaN
    
    # forecasting on n_steps forward 
    forecast = model.predict(start = dataframe.shape[0], end = dataframe.shape[0]+n_steps)
    forecast = dataframe.arima_model.append(forecast)
    # calculate error, again having shifted on s+d steps from the beginning
    error = mean_absolute_percentage_error(dataframe.actual[s+d:], dataframe.arima_model[s+d:])

    plt.figure(figsize=(15, 7))
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    plt.plot(forecast, color='r', label="model")
    plt.axvspan(dataframe.index[-1], forecast.index[-1], alpha=0.5, color='lightgrey')
    plt.plot(dataframe.actual, label="actual")
    plt.legend()
    plt.grid(True);

In [ ]:
# Forecasting of next 100 days
plotSARIMA(df.Close, best_model, 100)

In [ ]:
new_df = pd.DataFrame(df.Close.copy())
new_df.columns = ['y']

In [ ]:
for i in range(6,25):
    new_df['lag_{}'.format(i)] = new_df.y.shift(i)

In [ ]:
new_df.tail()

In [ ]:
# Prediction using linear model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def time_series_train_test_split(X,y,test_size):
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_train = y.iloc[:test_index]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [ ]:
X = new_df.dropna().drop(['y'],axis = 1)
y = new_df.dropna().y

In [ ]:
X_train, X_test, y_train, y_test = time_series_train_test_split(X,y,0.3)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)

In [ ]:
def plotModelResults(model, X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=False, 
                     plot_anomalies=False):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """
    
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)
        
        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_test))
            anomalies[y_test<lower] = y_test[y_test<lower]
            anomalies[y_test>upper] = y_test[y_test>upper]
            plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True);
    
def plotCoefficients(model):
    """
        Plots sorted coefficient values of the model
    """
    
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');

In [ ]:
plotModelResults(lr,plot_intervals=True)
plotCoefficients(lr)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(X_train.corr());

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV

ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)

plotModelResults(ridge, 
                 X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True, plot_anomalies=False)
plotCoefficients(ridge)

In [ ]:
lasso = LassoCV(cv=tscv)
lasso.fit(X_train_scaled, y_train)

plotModelResults(lasso, 
                 X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True, plot_anomalies=False)
plotCoefficients(lasso)

In [ ]:
from xgboost import XGBRegressor 

xgb = XGBRegressor()
xgb.fit(X_train_scaled, y_train)

In [ ]:
plotModelResults(xgb, 
                 X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True, plot_anomalies=True)

In [ ]:
# We conclude that -
# SARIMAX model forecasts the values with error of 2.!!% (Lowest)